In [1]:
import os
import torch
import openai
import chromadb
import warnings
import nest_asyncio
import pandas as pd

from llama_parse import LlamaParse
from llama_index.core import Document, VectorStoreIndex, get_response_synthesizer, StorageContext, QueryBundle
from llama_index.core.retrievers import VectorIndexRetriever, BaseRetriever
from llama_index.core.node_parser import SemanticDoubleMergingSplitterNodeParser, LanguageConfig
from llama_index.core.schema import NodeWithScore
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from typing import Literal, List, Optional
from dotenv import load_dotenv
from tqdm.auto import tqdm
from context_cite import ContextCiter

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
torch.cuda.set_device(1)

In [3]:
load_dotenv()
nest_asyncio.apply()
warnings.filterwarnings("ignore")

In [4]:
document = 'documents/attention.pdf'
MODEL_NAME = "Llama-3.2-1B-Instruct"
model_name = "meta-llama/Llama-3.2-1B-Instruct" # 3.2 1B Instruct for faster inference, 3.1 8B for better performance

In [5]:
openai.api_key = os.getenv("OPENAI_API_KEY")
    
config = LanguageConfig(language="english", spacy_model="en_core_web_md") # must download the model first
embed_model = OpenAIEmbedding()
splitter = SemanticDoubleMergingSplitterNodeParser(
    initial_threshold=0.6,
    appending_threshold=0.5,
    merging_threshold=0.6,
    language_config=config,
    max_chunk_size=1024,
)

In [6]:
# parser = LlamaParse(
#     api_key=os.getenv("LLAMA_CLOUD_API_TOKEN"),
#     num_workers=8,
#     show_progress=True,
#     result_type="markdown"
# )

In [7]:
# file = "documents/intro_to_ml.pdf"
# if not os.path.exists(file):
#     raise FileNotFoundError(f"File {file} not found")

In [8]:
# documents = parser.load_data(file)
# nodes = splitter.get_nodes_from_documents(documents, show_progress=True)
text = """
Attention Is All You Need.

Abstract.
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data.
1 Introduction.
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden states ht, as a function of the previous hidden state ht-1 and the input for position t. This inherently sequential nature precludes parallelization within training examples, which becomes critical at longer sequence lengths, as memory constraints limit batching across examples. Recent work has achieved significant improvements in computational efficiency through factorization tricks [21] and conditional computation [32], while also improving model performance in case of the latter. The fundamental constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.
"""
ts = text.split("\n")

nodes = [Document(text=t) for t in ts if len(t) > 0]

In [9]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)
vector_index = VectorStoreIndex(nodes=nodes, 
                         insert_batch_size=1024, 
                         storage_context=storage_context,
                         show_progress=True)                                                          

Generating embeddings:   0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
dense_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=5)
sparse_retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=5)

res_synth = get_response_synthesizer()

In [11]:
class HybridRetriever(BaseRetriever):
    def __init__(self, dense_retriever: BaseRetriever = dense_retriever, 
                 sparse_retriever: BaseRetriever = sparse_retriever,
                 mode: Literal["AND", "OR"] = "OR",
                 **kwargs) -> None:
        self.dense_retriever = dense_retriever
        self.sparse_retriever = sparse_retriever
        self.mode = mode

        super().__init__(**kwargs)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        dense_res = self.dense_retriever.retrieve(query_bundle)
        sparse_res = self.sparse_retriever.retrieve(query_bundle)

        dense_ids = {n.node.node_id for n in dense_res}
        sparse_ids = {n.node.node_id for n in sparse_res}

        combined_ids = {n.node.node_id: n for n in dense_res}
        combined_ids.update({n.node.node_id: n for n in sparse_res})

        if self.mode == "AND":
            ids = dense_ids.intersection(sparse_ids)

        elif self.mode == "OR":
            ids = dense_ids.union(sparse_ids)

        else:
            raise ValueError("Invalid mode. Must be either 'AND' or 'OR'.")
        
        retrieved_nodes = [combined_ids[id] for id in ids]
        return retrieved_nodes

In [12]:
hybrid_retriever = HybridRetriever(dense_retriever=dense_retriever, sparse_retriever=sparse_retriever)
query_engine = RetrieverQueryEngine(retriever=hybrid_retriever, response_synthesizer=res_synth)

In [13]:
def rerank(query, nodes,
           model=model_name,
           long=False,
           top_k=None):
    context = "\n\n".join([node.node.get_content() for node in nodes])
    
    cc = ContextCiter.from_pretrained(
        model_name,
        context=context,
        query=query,
        device="cuda:3",
        solver="elastic_net",
        num_ablations=128
    )

    attributions = cc.get_attributions(as_dataframe=True, top_k=len(nodes) if top_k is None else top_k)
    styler = attributions
    if hasattr(attributions, "data"):
        attributions_df = attributions.data
    else:
        attributions_df = attributions

    segments = cc.sources
    score_list = attributions_df["Score"].tolist()

    node_scores = {}
    for node in nodes:
        node_text = node.node.get_content()
        cumulative_score = 0.0
        for seg, score in zip(segments, score_list):
            if seg.strip() and seg.strip() in node_text:
                cumulative_score += score
        node_scores[node.node.node_id] = cumulative_score

    reranked_nodes = sorted(nodes, key=lambda x: node_scores.get(x.node.node_id, 0.0), reverse=True)
    return reranked_nodes, cc.response, node_scores, attributions, styler

In [14]:
def query_pdf(query: str, top_k: int = 5) -> list:
    """ 
    Query the PDF document using the query engine."
    """
    global query_engine, nodes
    if query_engine is None:
        return "Please upload a PDF first."

    long = True if len(nodes) > 100 else False
    rnodes = query_engine.retriever.retrieve(QueryBundle(query))
    reranked_nodes, cc_response, node_scores, attrs, styler = rerank(query, rnodes)
    
    top_nodes = reranked_nodes[:top_k]  
    final_context = "\n\n".join([node.node.get_content() for node in top_nodes])
    
    final_response = (
        f"{cc_response}\n\nTop Ranked Context:\n{final_context}\n\nNode Scores:\n{node_scores}"
    )

    
    if hasattr(attrs, "data"):
        attrs = attrs.data
    
    return final_response, attrs, styler

In [21]:
res, attributions, board = query_pdf("What type of GPUs did the authors use in this paper?")

Attributed: The authors used eight P100 GPUs in their experiments.


In [22]:
board

,Score,Source
0,16.017,The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.
1,3.368,"Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature."
2,2.937,Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train.
3,0.661,"Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in the input or output sequences [2, 19]."
4,0.585,"We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely."
5,0.416,Attention Is All You Need.
6,0.169,"In all but a few cases [27], however, such attention mechanisms are used in conjunction with a recurrent network."
